In [1]:
import pandas as pd

In [2]:
account = pd.read_csv('data/account_booking_train.csv')
account.drop_duplicates(subset='transaction_reference_id', inplace=True)
print(account.shape)
account.head()

(11264, 6)


,transaction_reference_id,debit_credit_indicator,account_id,transaction_amount,transaction_currency,transaction_date
0,0ace8fca6ada96883ef2e823b5dea26b,CREDIT,25110,5249.26,GBP,2023-05-15
1,d52c4f1a546f5d784ee46a8f347ad607,DEBIT,27293,4481.50,GBP,2023-02-27
2,dac45362e7471a7fa2726c81adae8534,DEBIT,23088,1347.27,GBP,2023-03-18
3,2ee574398cd6c4a7f3e111447141550e,DEBIT,21641,9276.56,GBP,2023-07-15
4,c5154ea99a0ff84ba8e72217d34d3397,CREDIT,24233,8002.28,GBP,2024-01-12


In [3]:
external = pd.read_csv('data/external_parties_train.csv')
print(external.shape)
external.head()

(11064, 14)


,transaction_reference_id,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id
0,04ff0d1c680189e3a80c92d86407f0f5,BENE,mary mith 107 107 angela brooks n. thomasfurt ...,mary mith,angela brooks,107 107,NaN,NaN,n. thomasfurt,NaN,bulgaria,GB49MYOB82127728573340,+1.815660-6791x8486,50039037
1,439ab0ad7380e6135ab2ff3fddd4a727,ORG,yesneia kim north michael 93971 koribati,yesneia kim,north michael,NaN,NaN,93971,koribati,NaN,NaN,NaN,0 (269)620-8734x2349,60044692
2,00cac12d41191a84f9e31aa731a83512,ORG,w. roberson jr. 41010 rachel crossingapt. 923 ...,w. roberson jr.,rachel crossingapt.,41010 923,NaN,p2235417,thompsonshire amyport,NaN,NaN,GB08OTHR53515837682953,NaN,30008244
3,e4fba5f878dd3453e35973605a783a16,BENE,azquez-nelson co. suarez ports suite & 024 bri...,azquez-nelson co.,ports suite &,NaN,NaN,NaN,brittanyberg,NaN,bulgaria bulgaria,GB17VVGW66321494633280,NaN,40017944
4,d03d7e4c31878b0255d39e8c3f0ab625,ORG,m.j. bytd iii 856 john lake s. glenn cocos (ke...,m.j. bytd iii,john lake s. glenn,856,NaN,125838276,cocos (keeling),islands,NaN,NaN,(260)3371534,40012658


In [4]:
df = pd.merge(account, external)
# df.set_index('transaction_reference_id', inplace=True)
df.head()

,transaction_reference_id,debit_credit_indicator,account_id,transaction_amount,transaction_currency,transaction_date,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id
0,0ace8fca6ada96883ef2e823b5dea26b,CREDIT,25110,5249.26,GBP,2023-05-15,ORG,graham-hardy co. 618 hensley trail apt. 693 ne...,graham-hardy co.,hensley trail,618,NaN,04504,new christopherside,NaN,romania,GB30DOUY51013886267890,NaN,35010896
1,d52c4f1a546f5d784ee46a8f347ad607,DEBIT,27293,4481.50,GBP,2023-02-27,BENE,jennyrobinson 273 jason club suite 792 stacyla...,jennyrobinson,jason club suite,273 792,NaN,03131,stacyland archipelago),british indian ocean territory (chagos,NaN,GB07JCYH58957395909319,(0041) 9477655328,35020711
2,dac45362e7471a7fa2726c81adae8534,DEBIT,23088,1347.27,GBP,2023-03-18,BENE,austin-coleman 944 lee gateway south patrickvi...,austin-coleman,lee gateway south,944,NaN,29747,patrickville,NaN,NaN,GB70DBTW73039738294272,NaN,35024310
3,2ee574398cd6c4a7f3e111447141550e,DEBIT,21641,9276.56,GBP,2023-07-15,BENE,young-turner lake carolynborough lake kathryn ...,young-turner,lake carolynborough lake,NaN,NaN,NaN,kathryn,NaN,eritrea,GB81ISHY85215830502792,004 1 +1-900626-2567x48643,10084859
4,c5154ea99a0ff84ba8e72217d34d3397,CREDIT,24233,8002.28,GBP,2024-01-12,ORG,dr. wilzoon michael 14022 austin alley suite 9...,dr. wilzoon michael,austin alley suite s. amandafort murphyhaven m...,14022 998,NaN,NaN,barthelemy,saint,NaN,NaN,NaN,20010362


In [5]:
df[df.external_id == 35024310]

,transaction_reference_id,debit_credit_indicator,account_id,transaction_amount,transaction_currency,transaction_date,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id
2,dac45362e7471a7fa2726c81adae8534,DEBIT,23088,1347.27,GBP,2023-03-18,BENE,austin-coleman 944 lee gateway south patrickvi...,austin-coleman,lee gateway south,944,NaN,29747,patrickville,NaN,NaN,GB70DBTW73039738294272,NaN,35024310
2289,614e75e11bdce7cb05857c013b59923a,DEBIT,22625,987.61,GBP,2024-02-18,BENE,austin-coleman 9008 traci flat apt. 846 s. and...,austin-coleman,traci flat apt. andreaburgh,9008,NaN,20540,s.,brunei darussalam,NaN,GB70DBTW73039738294272,(+41)001-60029 7-6358x20636,35024310
7631,0df70414c631cd5741e7cecd99b34d8b,CREDIT,26166,8254.10,GBP,2023-03-18,ORG,austin-coleman larryborough afghanistan,austin-coleman,NaN,NaN,NaN,NaN,larryborough,NaN,afghanistan,GB70DBTW73039738294272,0 001600 -297-635820636,35024310


In [6]:
from recordlinkage.preprocessing import phonetic


df["phonetic_parsed_name"] = phonetic(df["parsed_name"], "soundex")
df["phonetic_parsed_address_street_name"] = phonetic(df["parsed_address_street_name"], "soundex")
df["phonetic_parsed_address_city"] = phonetic(df["parsed_address_city"], "soundex")
df["initials"] = (df["parsed_name"].str[0]  + df["parsed_name"].str[-1])

df.head()

,transaction_reference_id,debit_credit_indicator,account_id,transaction_amount,transaction_currency,transaction_date,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,...,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id,phonetic_parsed_name,phonetic_parsed_address_street_name,phonetic_parsed_address_city,initials
0,0ace8fca6ada96883ef2e823b5dea26b,CREDIT,25110,5249.26,GBP,2023-05-15,ORG,graham-hardy co. 618 hensley trail apt. 693 ne...,graham-hardy co.,hensley trail,...,new christopherside,NaN,romania,GB30DOUY51013886267890,NaN,35010896,G656,H524,N262,g.
1,d52c4f1a546f5d784ee46a8f347ad607,DEBIT,27293,4481.50,GBP,2023-02-27,BENE,jennyrobinson 273 jason club suite 792 stacyla...,jennyrobinson,jason club suite,...,stacyland archipelago),british indian ocean territory (chagos,NaN,GB07JCYH58957395909319,(0041) 9477655328,35020711,J561,J252,S324,jn
2,dac45362e7471a7fa2726c81adae8534,DEBIT,23088,1347.27,GBP,2023-03-18,BENE,austin-coleman 944 lee gateway south patrickvi...,austin-coleman,lee gateway south,...,patrickville,NaN,NaN,GB70DBTW73039738294272,NaN,35024310,A235,L232,P362,an
3,2ee574398cd6c4a7f3e111447141550e,DEBIT,21641,9276.56,GBP,2023-07-15,BENE,young-turner lake carolynborough lake kathryn ...,young-turner,lake carolynborough lake,...,kathryn,NaN,eritrea,GB81ISHY85215830502792,004 1 +1-900626-2567x48643,10084859,Y523,L226,K365,yr
4,c5154ea99a0ff84ba8e72217d34d3397,CREDIT,24233,8002.28,GBP,2024-01-12,ORG,dr. wilzoon michael 14022 austin alley suite 9...,dr. wilzoon michael,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,NaN,NaN,20010362,D642,A235,B634,dl


In [7]:
df[df.external_id == 20010362]

,transaction_reference_id,debit_credit_indicator,account_id,transaction_amount,transaction_currency,transaction_date,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,...,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id,phonetic_parsed_name,phonetic_parsed_address_street_name,phonetic_parsed_address_city,initials
4,c5154ea99a0ff84ba8e72217d34d3397,CREDIT,24233,8002.28,GBP,2024-01-12,ORG,dr. wilzoon michael 14022 austin alley suite 9...,dr. wilzoon michael,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,NaN,NaN,20010362,D642,A235,B634,dl
243,0ea2584fbf22684f1c75f33ec2aa3e8f,CREDIT,28510,6568.87,GBP,2024-08-31,ORG,michael wulson ii 14022 austin alley suite 998...,michael wulson ii,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,NaN,0090(561)317-9341,20010362,M244,A235,B634,mi
671,ea9d848a99c1489ed4e5914e909cd641,DEBIT,25530,6450.75,GBP,2023-03-17,BENE,dr. michail jr. 14022 austin alley suite 998 s...,dr. michail jr.,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,GB30AIYL19953245864415,NaN,20010362,D652,A235,B634,d.
1262,0693000527c3c5efffd7f7b6809d33ec,CREDIT,20068,2998.58,GBP,2023-10-21,ORG,mrs. michael 14022 austin alley suite 998 s. a...,mrs. michael,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,GB68JBVE39499747305252,+43(552)54 5-69 86,20010362,M625,A235,B634,ml
2796,919140816b1704e3e9bba5df12a1bf3a,CREDIT,29457,10.95,GBP,2023-06-01,ORG,michael willson 14022 austin alley suite 998 s...,michael willson,austin alley suite s. amandafort murphyhaven m...,...,barthelemy,saint,NaN,NaN,NaN,20010362,M244,A235,B634,mn


In [8]:
import recordlinkage
from recordlinkage.index import Block, SortedNeighbourhood
import pandas as pd

In [9]:
indexer = Block('initials')
candidate_links = indexer.index(df)

In [10]:
compare = recordlinkage.Compare()

In [11]:
compare = recordlinkage.Compare()

compare.string('parsed_name', 'parsed_name', label='parsed_name')
compare.exact('phonetic_parsed_name', 'phonetic_parsed_name', label="phonetic_parsed_name")
compare.string('parsed_address_street_name', 'parsed_address_street_name', method='jarowinkler', label="parsed_address_street_name")
compare.string('phonetic_parsed_address_street_name', 'phonetic_parsed_address_street_name', method='jarowinkler', label="phonetic_parsed_address_street_name")
compare.string('parsed_address_state', 'parsed_address_state', method='jarowinkler', label="parsed_address_state")
compare.string('parsed_address_city', 'parsed_address_city', method='jarowinkler', label="parsed_address_city")
compare.string('parsed_address_country', 'parsed_address_country', method='jarowinkler', label="parsed_address_country")
compare.string("party_iban","party_iban", label="party_iban")

features = compare.compute(candidate_links, df)
features.tail()

parsed_name  phonetic_parsed_name  parsed_address_street_name  \
10566 7011     0.222222                     0                    0.525926   
8008  7030     1.000000                     1                    0.477233   
9062  7030     1.000000                     1                    0.486365   
      8008     1.000000                     1                    0.524284   
7930  7082     0.538462                     0                    0.544444   

            phonetic_parsed_address_street_name  parsed_address_state  \
10566 7011                             0.500000              0.000000   
8008  7030                             0.666667              0.000000   
9062  7030                             0.500000              0.000000   
      8008                             0.500000              0.000000   
7930  7082                             0.666667              0.564815   

            parsed_address_city  parsed_address_country  party_iban  
10566 7011             0.000000                     0.0    0.227273  
8008  7030             0.446825                     0.0    1.000000  
9062  7030             0.523810                     0.0    1.000000  
      8008             0.495238                     0.0    1.000000  
7930  7082             0.361111                     0.0    0.136364

In [ ]:
threshold = 0.80
predicted_matches = features[(features['parsed_name'] == 1) | (features['parsed_address_street_name'] == 1) | ((features['parsed_name'] > threshold) & (features['parsed_address_street_name'] > threshold))].index

# Normalize pairs to ensure (2, 7) == (7, 2)
def normalize_pair(pair):
    return tuple(sorted(pair))

# Step 5: Generate ground truth matches from `true_label`
true_links = set()
grouped = df.groupby('external_id')
for _, group in grouped:
    ids = group.index.tolist()
    true_links.update([normalize_pair((ids[i], ids[j])) for i in range(len(ids)) for j in range(i + 1, len(ids))])

# Normalize predicted matches
predicted_links = {normalize_pair(pair) for pair in predicted_matches}


# Step 6: Evaluate
true_positives = len(predicted_links & true_links)
false_positives = len(predicted_links - true_links)
false_negatives = len(true_links - predicted_links)

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Print metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Precision: 0.96
Recall: 0.67
F1 Score: 0.79


In [18]:
# Step 4: Cluster entities
clusters = pd.DataFrame(list(predicted_links), columns=['entity1', 'entity2'])

G = nx.Graph()
G.add_edges_from(clusters.values)
connected_components = list(nx.connected_components(G))

# Step 5: Assign cluster IDs
cluster_id = 0
entity_to_cluster = {}
for component in connected_components:
    for entity in component:
        entity_to_cluster[entity] = cluster_id
    cluster_id += 1

# Step 6: Prepare the submission file
# Map the index values to the actual transaction_reference_id
transaction_reference_id = df.index.to_series().map(df['transaction_reference_id'])
submission = pd.DataFrame(list(entity_to_cluster.items()), columns=['index', 'external_id'])
submission['transaction_reference_id'] = submission['index'].map(transaction_reference_id)
submission = submission[['transaction_reference_id', 'external_id']]

# Remove singletons if needed
submission = submission[submission.duplicated('external_id', keep=False)]

# Save to CSV
submission.to_csv('submission.csv', index=False)